In [21]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import scipy as sp
from math import sqrt

In [2]:
df = pd.read_excel('Nerlove1963.xlsx')

In [3]:
df.head()

,Cost,output,Plabor,Pcapital,Pfuel
0,0.082,2,2.09,183,17.9
1,0.661,3,2.05,174,35.1
2,0.990,4,2.05,171,35.1
3,0.315,4,1.83,166,32.2
4,0.197,5,2.12,233,28.6


In [9]:
#take logs to estimate Cobb-Douglas
df['ln_cost'] = np.log(df['Cost'])
df['ln_output'] = np.log(df['output'])
df['ln_plabor'] = np.log(df['Plabor'])
df['ln_pcapital'] = np.log(df['Pcapital'])
df['ln_pfuel'] = np.log(df['Pfuel'])
df['intercept'] = 1

In [10]:
model = sm.OLS(df['ln_cost'], df[['ln_output', 'ln_plabor', 'ln_pcapital', 'ln_pfuel', 'intercept']])

In [11]:
res = model.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                ln_cost   R-squared:                       0.926
Model:                            OLS   Adj. R-squared:                  0.924
Method:                 Least Squares   F-statistic:                     437.7
Date:                Tue, 24 May 2022   Prob (F-statistic):           4.82e-78
Time:                        11:24:43   Log-Likelihood:                -67.542
No. Observations:                 145   AIC:                             145.1
Df Residuals:                     140   BIC:                             160.0
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
ln_output       0.7204      0.017     41.244      

In [12]:
#adding restriction
df['ln_pcap_plab'] = df['ln_pcapital'] - df['ln_plabor']
df['ln_pfuel_plab'] = df['ln_pfuel'] - df['ln_plabor']

In [13]:
model2 = sm.OLS(df['ln_cost'], df[['ln_output', 'ln_plabor', 'ln_pcap_plab', 'ln_pfuel_plab', 'intercept']])

In [14]:
res2 = model2.fit()
print(res2.summary())

                            OLS Regression Results                            
Dep. Variable:                ln_cost   R-squared:                       0.926
Model:                            OLS   Adj. R-squared:                  0.924
Method:                 Least Squares   F-statistic:                     437.7
Date:                Tue, 24 May 2022   Prob (F-statistic):           4.82e-78
Time:                        11:31:27   Log-Likelihood:                -67.542
No. Observations:                 145   AIC:                             145.1
Df Residuals:                     140   BIC:                             160.0
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
ln_output         0.7204      0.017     41.244

In [15]:
#t-stat
print(sp.stats.t.ppf(0.05, 140))

-1.6558105109932533


In [16]:
print(1 - sp.stats.t.sf(-0.758, 140))

0.2248625202283664


In [17]:
n = 145
k=5
x = df[['ln_output', 'ln_plabor', 'ln_pcapital', 'ln_pfuel', 'intercept']].to_numpy()
y = df[['ln_cost']].to_numpy()
beta = (np.linalg.inv((x.T)@x)@(x.T)@y).reshape(k,)
qxx = ((x.T)@x)*(1/n)
eh = (y - x@(beta.reshape(k,1))).reshape(n,)
xxe = x*(eh.reshape(n,1))
vxe = (xxe.T@xxe)*(1/n)
vbeta = (np.linalg.inv(qxx))@vxe@(np.linalg.inv(qxx))/n
R = np.array([0,1,1,1,0]).reshape(1,k)
W = (beta[1] + beta[2] + beta[3] - 1)*(beta[1] + beta[2] + beta[3] - 1)*n/((np.linalg.inv(R@vbeta@R.T)).item())

In [18]:
#Wald
W

3.52427091201883

In [19]:
print(sp.stats.chi2.ppf(0.95, 1))

3.841458820694124


In [20]:
print(sp.stats.chi2.sf(3.52427, 1))

0.060476451567195985


In [22]:
n = 145
k=5
x = df[['ln_output', 'ln_plabor', 'ln_pcap_plab', 'ln_pfuel_plab', 'intercept']].to_numpy() 
y = df[['ln_cost']].to_numpy()
beta = (np.linalg.inv((x.T)@x)@(x.T)@y).reshape(k,)
qxx = ((x.T)@x)*(1/n)
eh = (y - x@(beta.reshape(k,1))).reshape(n,)
xxe = x*(eh.reshape(n,1))
vxe = (xxe.T@xxe)*(1/n)
vbeta = (np.linalg.inv(qxx))@vxe@(np.linalg.inv(qxx))/n
T = (beta[1] - 1)/sqrt(vbeta[1,1])

In [23]:
T

-0.8176345924152044

In [24]:
print(1 - sp.stats.t.sf(-0.81763459, 140))

0.20747760625723943


**Comments** \
Economically, the restriction on betas means that the cost function exhibits constant return to scale wrt to cost of capital, labor, fuel. Equivalently, it means that the function is homogeneous of degree 1 in prices. At the same time, it should be taken into account that this restriction does not imply that prodiction function exhibits constant return to scale, since, according to theory, cost function has its price coefficients summed up to 1 even if Cobb-Douglas production function is not CRS. \
*As for the tests*. Left-sided test: t=-0,8176>-1,656. Hence, $H_0$ is not rejected at 5% level against $H_1$. 
Two-sided test: $W$ = 3,53 > $X^{1}_{0,95}$ = 3,84. $H_0$ is not rejected at 5% level. \
*p-value*: in the first case the p-value = 0,225. It indicates whether $H_0$ : $\beta_3 + \beta_4 + \beta_5 <1$ can not be rejected. 
In the second case, p-value is 0,06. It indicates whether $H_0$ : $\beta_3 + \beta_4 + \beta_5 =1$ can not be rejected. Hence, due to the construction and logic, they are responsible for different $H_1$. The distribution of Wald statistic is symmetric and not one-sided. t-test is not symmetric, it can be either 1-sided or 2-sided, we can choose the side we want or even both. 